In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud ,STOPWORDS
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

import re
import warnings
warnings.filterwarnings("ignore")

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
import seaborn as sns
sns.set_style("darkgrid")
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import tensorflow as tf
#import tensorflow_hub as hub
#import tensorflow_text as text

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, GlobalMaxPool1D, Embedding, Activation, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from keras.models import Sequential
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from sklearn import preprocessing

In [6]:
tf.device('/gpu:0')

In [7]:
df = pd.read_csv("/content/drive/MyDrive/OSTI/Classification/binary/dataset/Final_OSTI_Binary_12_12_2023.csv",encoding='utf-8').dropna()

In [8]:
df['Domain'] = df['Domain'].apply(int)

In [9]:
df['Domain'].value_counts()

0    6341
1    3671
Name: Domain, dtype: int64

In [10]:
# Remove duplicate records based on all columns
df = df.drop_duplicates()

In [11]:
df.shape

(10012, 3)

# Preprocessing

In [12]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
def replaceDegfanged(text):
    """ Replaces url address with "url" """
    text = re.sub('([A-Za-z]+://)([-\w]+(?:\.\w[-\w]*)+)(:\d+)?(/[^.!,?;"\x27<>()\[\]{}\s\x7F-\xFF]*(?:[.!,?]+[^.!,?;"\x27<>()\[\]{}\s\x7F-\xFF]+)*)?','',text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    return text

def replaceURL(text):
    """ Replaces url address with "url" """
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    return text

def remove_emoji(string):
    if type(string) != str:
        return ''
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('A'):
        return wordnet.ADJ
    elif treebank_tag.startswith('S'):
        return wordnet.ADJ_SAT
    elif treebank_tag.startswith('SYM'):
        return wordnet.SYM
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    elif treebank_tag.startswith('WRB'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


In [14]:
from nltk.stem import WordNetLemmatizer
wl = WordNetLemmatizer()

In [15]:
def preprocessing_df(text):
  tweets = []
  for sentence in tqdm(text):
    sentence = sentence.lower() # converting the words to lower case
    # Remove newline characters using regular expression
    # Remove newline characters and extra spaces using regular expression
    sentence = re.sub(r'\s+', ' ', sentence.replace('\n', ' ')).strip()
    sentence = re.sub(r'https://t.co/\S+','', sentence)
    sentence =  replaceURL(sentence) # replace the url from the sentence
    sentence = replaceDegfanged(sentence)
    sentence = re.sub(r'hxxp\S+','', sentence)
    sentence = re.sub('CVE-\d{4}-\d{4,7}','',sentence)
    sentence = sentence.replace('dot', '.')
    sentence = sentence.replace('[.]', '.')
    sentence = re.sub(r"[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}", "",sentence)
    sentence = re.sub(r"\b^\$2[ayb]\$.{56}$\b",'',sentence) #hash 256
    sentence = re.sub(r"\b[a-f0-9]{40}|[A-F0-9]{40}\b",'',sentence) #hash 1
    sentence = re.sub(r"\b[a-f0-9]{32}|[A-F0-9]{32}\b",'',sentence) #md5
    sentence = re.sub(r"\b[a-z0-9]+@[a-z]+\.[a-z]{2,3}\b","",sentence)
    sentence = remove_emoji(sentence)
    sentence = re.sub("(?<=\.)([^.]+)(?:\.(?:co\.uk|ac\.us|com|net|org|info|coop|int|co|ac|ie|co|ai|eu|ca|icu|top|xyz|tk|cn|ga|cf|nl|us|eu|de|hk|am|tv|bingo|blackfriday|gov|edu|mil|arpa|au|ru|[^.]+(?:$|\n)))",'',sentence)
    sentence = sentence.replace('.', '. ')
    sentence = re.sub("[^a-zA-Z ']", "", sentence) # removing symbols
    sentence = ' '.join( [w for w in sentence.split() if len(w)>1] ) #single charcater removal

    #stopword removal
    sentence = sentence.split()
    word_and_tag = nltk.pos_tag(sentence)
    sentence = [wl.lemmatize(word,pos=get_wordnet_pos(tag)) for word,tag in word_and_tag if word not in set(stopwords.words("english"))]
    sentence = " ".join(sentence)

    tweets.append(sentence)
  return tweets
df['preprocess_description'] = preprocessing_df(df['Description'])

100%|██████████| 10012/10012 [21:43<00:00,  7.68it/s]


In [16]:
df

,Domain,Title,Description,preprocess_description
0,1,DISH slapped with multiple lawsuits after rans...,Dish Network has been slapped with multiple cl...,dish network slap multiple class action lawsui...
1,1,Bing search results hijacked via misconfigured...,A misconfigured Microsoftapplication allowed a...,misconfigured microsoftapplication allow anyon...
2,1,Crown Resorts confirms ransom demand after GoA...,"Crown Resorts, Australia's largest gambling an...",crown resort australia's large gambling entert...
3,1,New MacStealer macOS malware steals passwords ...,A new info-stealing malware named MacStealer i...,new infostealing malware name macstealer targe...
4,1,"Hackers earn $1,035,000 for 27 zero-days explo...",Pwn2Own Vancouver 2023 has ended with contesta...,pwnown vancouver end contestant earn tesla mod...
...,...,...,...,...
10008,0,Microsoft Patch Tuesday for August 2021 — Snor...,"By Jon Munshaw, with contributions from Martin...",jon munshaw contribution martin lee microsoft ...
10009,0,Vulnerability Spotlight: Multiple vulnerabilit...,Carl Hurd of Cisco Talos discovered these vuln...,carl hurd cisco talos discover vulnerability c...
10010,0,Talos Incident Response quarterly threat repor...,By David Liebenberg and Caitlin Huey.Last quar...,david liebenberg caitlin huey last quarter ran...
10011,0,Vice Society leverages PrintNightmare in ranso...,Executive Summary Another threat actor is acti...,executive summary another threat actor activel...


In [17]:
# saving the dataframe
df.to_csv('/content/drive/MyDrive/OSTI/Classification/binary/dataset/Preprocessed_Final_OSTI_Binary_12_12_2023.csv',index=False)

In [ ]:
import re

paragraph = """This is a    sample    paragraph.
It spans multiple   lines.
We want to remove     the newline characters and extra spaces."""

# Remove newline characters and extra spaces using regular expression
cleaned_paragraph = re.sub(r'\s+', ' ', paragraph.replace('\n', ' ')).strip()

print(cleaned_paragraph)


This is a sample paragraph. It spans multiple lines. We want to remove the newline characters and extra spaces.


In [ ]:
import random

dataset_size = 10612
num_seeds = 10

# Generate 10 random seed values
random_seeds = [random.randint(1, 10612) for _ in range(num_seeds)]

# Print the generated seed values
print("Random Seed Values:", random_seeds)

Random Seed Values: [3832, 479, 3153, 6169, 8891, 1761, 6025, 7871, 1659, 7604]
